In [ ]:
!pip install yfinance scikit-learn tensorflow pandas numpy matplotlib gradio ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=744a3c8732dd728e5eef6befea4e7f3cc110415f671af4d0ef920ba446b1da6e
  Stored in directory: /root/.cache/pip/wheels/5c/a1/5f/c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta


In [7]:
# # FULLY DEBUGGED Stock Predictor - LSTM + Bidirectional LSTM Only
# # Comprehensive error handling and logging built-in

# import os, time, threading, traceback, warnings
# import numpy as np
# import pandas as pd
# import yfinance as yf
# from sklearn.preprocessing import MinMaxScaler
# import tensorflow as tf
# from tensorflow.keras import layers, models, callbacks
# import matplotlib
# matplotlib.use('Agg')  # Non-interactive backend
# import matplotlib.pyplot as plt
# import gradio as gr

# # Suppress warnings
# warnings.filterwarnings('ignore')
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# # ---------------- CONFIG ----------------
# TOP_10_STOCKS = ["AAPL","MSFT","AMZN","GOOGL","META","TSLA","NVDA","NFLX","JPM","AMD"]
# MODELS_DIR = "/content/stock_models"
# os.makedirs(MODELS_DIR, exist_ok=True)

# # OPTIMIZED FOR SPEED
# SEQ_LEN = 20
# POLL_INTERVAL = 10
# EPOCHS = 3
# BATCH_SIZE = 64

# # Shared state
# LIVE_STATE = {}
# POLLERS = {}
# TRAIN_LOGS = {}
# TRAINING_FLAGS = {}

# # ---------------- LOGGING UTILITY ----------------
# class Logger:
#     def __init__(self, symbol):
#         self.symbol = symbol
#         self.logs = []

#     def log(self, msg, level="INFO"):
#         timestamp = time.strftime("%H:%M:%S")
#         emoji = {"INFO": "ℹ️", "SUCCESS": "✅", "ERROR": "❌", "WARNING": "⚠️"}.get(level, "📝")
#         line = f"[{timestamp}] {emoji} {msg}"
#         self.logs.append(line)
#         print(line)
#         TRAIN_LOGS[self.symbol] = "\n".join(self.logs)
#         return line

# # ---------------- DATA FUNCTIONS ----------------
# def download_stock_data(symbol, period='1y', logger=None):
#     """Download stock data with error handling"""
#     try:
#         if logger:
#             logger.log(f"Downloading {symbol} data ({period})...")

#         df = yf.download(symbol, period=period, interval='1d',
#                         progress=False, auto_adjust=True, threads=False)

#         if df.empty:
#             raise ValueError(f"No data returned for {symbol}")

#         # Handle multi-level columns
#         if isinstance(df.columns, pd.MultiIndex):
#             df.columns = df.columns.get_level_values(0)

#         # Keep only OHLCV
#         required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
#         df = df[required_cols].copy()

#         # Drop NaN
#         df = df.dropna()

#         if logger:
#             logger.log(f"Downloaded {len(df)} rows", "SUCCESS")

#         return df

#     except Exception as e:
#         if logger:
#             logger.log(f"Download failed: {str(e)}", "ERROR")
#         raise

# def add_technical_indicators(df, logger=None):
#     """Add simple technical indicators without external libraries"""
#     try:
#         if logger:
#             logger.log("Adding technical indicators...")

#         # Ensure Close is 1D
#         close = df['Close'].values.flatten()

#         # Simple Moving Average
#         df['SMA_10'] = pd.Series(close).rolling(window=10).mean()

#         # Exponential Moving Average
#         df['EMA_10'] = pd.Series(close).ewm(span=10, adjust=False).mean()

#         # RSI (Relative Strength Index) - manual calculation
#         delta = pd.Series(close).diff()
#         gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
#         loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
#         rs = gain / loss
#         df['RSI_14'] = 100 - (100 / (1 + rs))

#         # Volume change
#         df['Volume_Change'] = pd.Series(df['Volume'].values).pct_change()

#         # Drop NaN from indicators
#         df = df.dropna()

#         if logger:
#             logger.log(f"Added indicators, {len(df)} rows remaining", "SUCCESS")

#         return df

#     except Exception as e:
#         if logger:
#             logger.log(f"Indicator calculation failed: {str(e)}", "ERROR")
#         raise

# def prepare_sequences(df, seq_len=SEQ_LEN, logger=None):
#     """Prepare sequences for training"""
#     try:
#         if logger:
#             logger.log("Preparing sequences...")

#         # Feature columns
#         feature_cols = ['Close', 'SMA_10', 'EMA_10', 'RSI_14', 'Volume_Change']

#         # Verify all columns exist
#         missing = [col for col in feature_cols if col not in df.columns]
#         if missing:
#             raise ValueError(f"Missing columns: {missing}")

#         # Get values and scale
#         arr = df[feature_cols].values.astype(np.float32)

#         scaler = MinMaxScaler()
#         scaled = scaler.fit_transform(arr)

#         # Create sequences
#         X, y = [], []
#         for i in range(seq_len, len(scaled)):
#             X.append(scaled[i-seq_len:i])
#             y.append(scaled[i, 0])  # Predict Close price

#         X = np.array(X, dtype=np.float32)
#         y = np.array(y, dtype=np.float32)

#         if logger:
#             logger.log(f"Created {len(X)} sequences", "SUCCESS")

#         return X, y, scaler, len(feature_cols)

#     except Exception as e:
#         if logger:
#             logger.log(f"Sequence preparation failed: {str(e)}", "ERROR")
#         raise

# # ---------------- MODEL ARCHITECTURES ----------------
# def build_lstm_model(seq_len, n_features):
#     """Simple LSTM model"""
#     model = models.Sequential([
#         layers.Input(shape=(seq_len, n_features)),
#         layers.LSTM(64, return_sequences=True),
#         layers.Dropout(0.2),
#         layers.LSTM(32),
#         layers.Dense(16, activation='relu'),
#         layers.Dense(1)
#     ])
#     model.compile(
#         optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#         loss='mse',
#         metrics=['mae']
#     )
#     return model

# def build_bidirectional_lstm_model(seq_len, n_features):
#     """Bidirectional LSTM model"""
#     model = models.Sequential([
#         layers.Input(shape=(seq_len, n_features)),
#         layers.Bidirectional(layers.LSTM(64, return_sequences=True)),
#         layers.Dropout(0.2),
#         layers.Bidirectional(layers.LSTM(32)),
#         layers.Dense(16, activation='relu'),
#         layers.Dense(1)
#     ])
#     model.compile(
#         optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#         loss='mse',
#         metrics=['mae']
#     )
#     return model

# # ---------------- MODEL SAVE/LOAD ----------------
# def save_model_safe(model, path, logger=None):
#     """Save model in TF SavedModel format"""
#     try:
#         save_path = path.replace('.h5', '_saved_model')
#         model.save(save_path, save_format='tf')
#         if logger:
#             logger.log(f"Model saved to {save_path}", "SUCCESS")
#         return save_path
#     except Exception as e:
#         if logger:
#             logger.log(f"Save failed: {str(e)}", "ERROR")
#         raise

# def load_model_safe(path, logger=None):
#     """Load model from SavedModel format"""
#     try:
#         load_path = path.replace('.h5', '_saved_model')
#         if not os.path.exists(load_path):
#             raise FileNotFoundError(f"Model not found at {load_path}")

#         model = tf.keras.models.load_model(load_path)
#         if logger:
#             logger.log(f"Model loaded from {load_path}", "SUCCESS")
#         return model
#     except Exception as e:
#         if logger:
#             logger.log(f"Load failed: {str(e)}", "ERROR")
#         return None

# # ---------------- TRAINING THREAD ----------------
# def train_models_thread(symbol):
#     """Background training with comprehensive error handling"""
#     TRAINING_FLAGS[symbol] = True
#     logger = Logger(symbol)

#     try:
#         logger.log(f"🚀 Starting training for {symbol}")

#         # Step 1: Download data
#         df = download_stock_data(symbol, period='1y', logger=logger)

#         # Step 2: Add indicators
#         df = add_technical_indicators(df, logger=logger)

#         # Step 3: Prepare sequences
#         X, y, scaler, n_features = prepare_sequences(df, SEQ_LEN, logger=logger)

#         # Check data size
#         if len(X) < 100:
#             raise ValueError(f"Insufficient data: {len(X)} samples (need 100+)")

#         # Step 4: Train/test split
#         split_idx = int(len(X) * 0.8)
#         X_train, X_test = X[:split_idx], X[split_idx:]
#         y_train, y_test = y[:split_idx], y[split_idx:]

#         logger.log(f"Training: {len(X_train)} samples, Testing: {len(X_test)} samples")

#         # Step 5: Train LSTM
#         logger.log("🏗️ Building LSTM model...")
#         lstm_model = build_lstm_model(SEQ_LEN, n_features)

#         logger.log(f"🎓 Training LSTM ({EPOCHS} epochs)...")
#         history_lstm = lstm_model.fit(
#             X_train, y_train,
#             validation_data=(X_test, y_test),
#             epochs=EPOCHS,
#             batch_size=BATCH_SIZE,
#             verbose=0,
#             callbacks=[
#                 callbacks.EarlyStopping(patience=2, restore_best_weights=True)
#             ]
#         )

#         final_loss = history_lstm.history['val_loss'][-1]
#         logger.log(f"LSTM trained! Val Loss: {final_loss:.4f}", "SUCCESS")

#         # Save LSTM
#         lstm_path = os.path.join(MODELS_DIR, f"{symbol}_lstm.h5")
#         save_model_safe(lstm_model, lstm_path, logger)

#         # Step 6: Train Bidirectional LSTM
#         logger.log("🏗️ Building Bidirectional LSTM model...")
#         bilstm_model = build_bidirectional_lstm_model(SEQ_LEN, n_features)

#         logger.log(f"🎓 Training Bidirectional LSTM ({EPOCHS} epochs)...")
#         history_bilstm = bilstm_model.fit(
#             X_train, y_train,
#             validation_data=(X_test, y_test),
#             epochs=EPOCHS,
#             batch_size=BATCH_SIZE,
#             verbose=0,
#             callbacks=[
#                 callbacks.EarlyStopping(patience=2, restore_best_weights=True)
#             ]
#         )

#         final_loss = history_bilstm.history['val_loss'][-1]
#         logger.log(f"Bidirectional LSTM trained! Val Loss: {final_loss:.4f}", "SUCCESS")

#         # Save Bidirectional LSTM
#         bilstm_path = os.path.join(MODELS_DIR, f"{symbol}_bilstm.h5")
#         save_model_safe(bilstm_model, bilstm_path, logger)

#         # Step 7: Save scaler
#         scaler_path = os.path.join(MODELS_DIR, f"{symbol}_scaler.npz")
#         np.savez(scaler_path,
#                  min=scaler.min_,
#                  scale=scaler.scale_,
#                  feature_names=np.array(['Close', 'SMA_10', 'EMA_10', 'RSI_14', 'Volume_Change']))
#         logger.log("Scaler saved", "SUCCESS")

#         logger.log("🎉 Training completed successfully!")
#         logger.log(f"⏱️ Models ready for {symbol}")

#     except Exception as e:
#         logger.log(f"Training failed: {str(e)}", "ERROR")
#         logger.log("Traceback:", "ERROR")
#         for line in traceback.format_exc().split('\n'):
#             if line.strip():
#                 logger.log(line, "ERROR")

#     finally:
#         TRAINING_FLAGS[symbol] = False

# # ---------------- HELPER FUNCTIONS ----------------
# def is_trained(symbol):
#     """Check if models exist"""
#     lstm_path = os.path.join(MODELS_DIR, f"{symbol}_lstm_saved_model")
#     bilstm_path = os.path.join(MODELS_DIR, f"{symbol}_bilstm_saved_model")
#     scaler_path = os.path.join(MODELS_DIR, f"{symbol}_scaler.npz")

#     return (os.path.exists(lstm_path) and
#             os.path.exists(bilstm_path) and
#             os.path.exists(scaler_path))

# def load_models_and_scaler(symbol):
#     """Load both models and scaler"""
#     logger = Logger(symbol)

#     lstm_path = os.path.join(MODELS_DIR, f"{symbol}_lstm.h5")
#     bilstm_path = os.path.join(MODELS_DIR, f"{symbol}_bilstm.h5")
#     scaler_path = os.path.join(MODELS_DIR, f"{symbol}_scaler.npz")

#     lstm_model = load_model_safe(lstm_path, logger)
#     bilstm_model = load_model_safe(bilstm_path, logger)

#     if lstm_model is None or bilstm_model is None:
#         raise RuntimeError("Failed to load models")

#     # Load scaler
#     scaler_data = np.load(scaler_path)
#     scaler = MinMaxScaler()
#     scaler.min_ = scaler_data['min']
#     scaler.scale_ = scaler_data['scale']

#     return lstm_model, bilstm_model, scaler

# def ensemble_predict(lstm_model, bilstm_model, X):
#     """Ensemble prediction from both models"""
#     pred_lstm = lstm_model.predict(X, verbose=0).flatten()
#     pred_bilstm = bilstm_model.predict(X, verbose=0).flatten()
#     return (pred_lstm + pred_bilstm) / 2.0

# # ---------------- UI FUNCTIONS ----------------
# def start_training(symbol):
#     """Start training in background"""
#     if TRAINING_FLAGS.get(symbol, False):
#         return f"⚠️ Training already running for {symbol}"

#     thread = threading.Thread(target=train_models_thread, args=(symbol,), daemon=True)
#     thread.start()

#     return f"🚀 Started training for {symbol}. Check logs below..."

# def get_training_logs(symbol):
#     """Retrieve training logs"""
#     if symbol in TRAINING_FLAGS and TRAINING_FLAGS[symbol]:
#         status = "🔄 Training in progress...\n\n"
#     else:
#         status = "✅ Training completed (or not started)\n\n"

#     logs = TRAIN_LOGS.get(symbol, "No logs yet. Click 'Train Models' to start.")
#     return status + logs

# def make_prediction(symbol):
#     """Make a single prediction"""
#     try:
#         if not is_trained(symbol):
#             return f"❌ Models not trained for {symbol}. Train first!"

#         # Load models
#         lstm_model, bilstm_model, scaler = load_models_and_scaler(symbol)

#         # Get recent data
#         df = download_stock_data(symbol, period='3mo')
#         df = add_technical_indicators(df)

#         X, y, _, _ = prepare_sequences(df, SEQ_LEN)

#         if len(X) == 0:
#             return "❌ Insufficient data for prediction"

#         # Get last sequence
#         last_seq = X[-1:]

#         # Predict
#         pred_scaled = ensemble_predict(lstm_model, bilstm_model, last_seq)[0]

#         # Inverse transform
#         dummy = np.zeros((1, scaler.scale_.shape[0]))
#         dummy[0, 0] = pred_scaled
#         pred_price = scaler.inverse_transform(dummy)[0, 0]

#         # Current price
#         current_price = df['Close'].values[-1]
#         change_pct = ((pred_price - current_price) / current_price) * 100

#         result = f"""📊 **Prediction for {symbol}**

# Current Price: ${current_price:.2f}
# Predicted Next: ${pred_price:.2f}
# Expected Change: {change_pct:+.2f}%

# 🤖 Ensemble: LSTM + Bidirectional LSTM
# """
#         return result

#     except Exception as e:
#         return f"❌ Error: {str(e)}"

# def get_validation_plot(symbol):
#     """Generate validation plot"""
#     try:
#         if not is_trained(symbol):
#             fig = plt.figure(figsize=(10, 4))
#             plt.text(0.5, 0.5, f"Models not trained for {symbol}\nClick 'Train Models' first",
#                     ha='center', va='center', fontsize=14)
#             plt.axis('off')
#             return fig

#         # Load models
#         lstm_model, bilstm_model, scaler = load_models_and_scaler(symbol)

#         # Get data
#         df = download_stock_data(symbol, period='6mo')
#         df = add_technical_indicators(df)
#         X, y, _, _ = prepare_sequences(df, SEQ_LEN)

#         # Use last 20% as test
#         split_idx = int(len(X) * 0.8)
#         X_test = X[split_idx:]
#         y_test = y[split_idx:]

#         # Predict
#         pred_scaled = ensemble_predict(lstm_model, bilstm_model, X_test)

#         # Inverse transform
#         dummy_pred = np.zeros((len(pred_scaled), scaler.scale_.shape[0]))
#         dummy_pred[:, 0] = pred_scaled
#         pred_prices = scaler.inverse_transform(dummy_pred)[:, 0]

#         dummy_actual = np.zeros((len(y_test), scaler.scale_.shape[0]))
#         dummy_actual[:, 0] = y_test
#         actual_prices = scaler.inverse_transform(dummy_actual)[:, 0]

#         # Plot
#         fig = plt.figure(figsize=(12, 5))
#         plt.plot(actual_prices, label='Actual', linewidth=2, color='blue')
#         plt.plot(pred_prices, label='Predicted', linewidth=2, color='red', alpha=0.7)
#         plt.title(f'{symbol} - Validation: Actual vs Predicted', fontsize=14, fontweight='bold')
#         plt.xlabel('Time Steps')
#         plt.ylabel('Price ($)')
#         plt.legend()
#         plt.grid(True, alpha=0.3)
#         plt.tight_layout()

#         return fig

#     except Exception as e:
#         fig = plt.figure(figsize=(10, 4))
#         plt.text(0.5, 0.5, f"Error generating plot:\n{str(e)}",
#                 ha='center', va='center', fontsize=12)
#         plt.axis('off')
#         return fig

# def download_model_files(symbol):
#     """Provide download links for models"""
#     if not is_trained(symbol):
#         return None, None, None

#     lstm_path = os.path.join(MODELS_DIR, f"{symbol}_lstm_saved_model")
#     bilstm_path = os.path.join(MODELS_DIR, f"{symbol}_bilstm_saved_model")
#     scaler_path = os.path.join(MODELS_DIR, f"{symbol}_scaler.npz")

#     # Create zip files for SavedModel directories
#     import shutil
#     lstm_zip = f"{lstm_path}.zip"
#     bilstm_zip = f"{bilstm_path}.zip"

#     if not os.path.exists(lstm_zip):
#         shutil.make_archive(lstm_path, 'zip', lstm_path)
#     if not os.path.exists(bilstm_zip):
#         shutil.make_archive(bilstm_path, 'zip', bilstm_path)

#     return lstm_zip, bilstm_zip, scaler_path

# # ---------------- GRADIO INTERFACE ----------------
# with gr.Blocks(theme=gr.themes.Soft(), title="Stock Predictor") as app:
#     gr.Markdown("""
#     # 📈 Stock Price Predictor
#     ### LSTM + Bidirectional LSTM Ensemble Model
#     Train models on historical data and predict future stock prices
#     """)

#     with gr.Row():
#         stock_dropdown = gr.Dropdown(
#             choices=TOP_10_STOCKS,
#             value="AAPL",
#             label="📊 Select Stock",
#             interactive=True
#         )
#         train_button = gr.Button("🚀 Train Models", variant="primary", size="lg")

#     with gr.Row():
#         logs_display = gr.Textbox(
#             label="📋 Training Logs",
#             lines=12,
#             max_lines=20,
#             interactive=False
#         )
#         refresh_logs_btn = gr.Button("🔄 Refresh Logs")

#     gr.Markdown("---")
#     gr.Markdown("## 🎯 Make Predictions")

#     with gr.Row():
#         predict_button = gr.Button("🔮 Predict Next Price", variant="secondary", size="lg")
#         prediction_output = gr.Textbox(
#             label="Prediction Result",
#             lines=7,
#             interactive=False
#         )

#     with gr.Row():
#         validation_plot = gr.Plot(label="📊 Validation Plot (Actual vs Predicted)")

#     gr.Markdown("---")
#     gr.Markdown("## 💾 Download Trained Models")

#     with gr.Row():
#         lstm_download = gr.File(label="LSTM Model (ZIP)")
#         bilstm_download = gr.File(label="Bidirectional LSTM Model (ZIP)")
#         scaler_download = gr.File(label="Scaler Data")

#     # Event handlers
#     train_button.click(
#         fn=start_training,
#         inputs=[stock_dropdown],
#         outputs=[logs_display]
#     )

#     refresh_logs_btn.click(
#         fn=get_training_logs,
#         inputs=[stock_dropdown],
#         outputs=[logs_display]
#     )

#     predict_button.click(
#         fn=make_prediction,
#         inputs=[stock_dropdown],
#         outputs=[prediction_output]
#     )

#     stock_dropdown.change(
#         fn=get_validation_plot,
#         inputs=[stock_dropdown],
#         outputs=[validation_plot]
#     )

#     stock_dropdown.change(
#         fn=download_model_files,
#         inputs=[stock_dropdown],
#         outputs=[lstm_download, bilstm_download, scaler_download]
#     )

# # Launch app
# if __name__ == "__main__":
#     print("=" * 50)
#     print("🚀 Stock Predictor App Starting...")
#     print("=" * 50)
#     app.launch(share=True, debug=False)

🚀 Stock Predictor App Starting...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://31de3ab31abc668c48.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [8]:
# # Full Gradio stock predictor app (LSTM + Transformer ensemble, indicators, polling live prices)
# # FIXED VERSION - All errors resolved

# import os, time, threading, io, traceback
# import numpy as np
# import pandas as pd
# import yfinance as yf
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.model_selection import train_test_split
# import tensorflow as tf
# from tensorflow.keras import layers, models, callbacks
# import matplotlib.pyplot as plt
# import gradio as gr
# import ta

# # ---------------- CONFIG ----------------
# TOP_10_STOCKS = ["AAPL","MSFT","AMZN","GOOGL","META","TSLA","NVDA","NFLX","JPM","AMD"]
# MODELS_DIR = "/content/stock_models"
# os.makedirs(MODELS_DIR, exist_ok=True)

# SEQ_LEN = 20             # ULTRA-FAST: Reduced from 60
# POLL_INTERVAL = 8        # seconds for live polling
# EPOCHS_REG = 2           # ULTRA-FAST: Minimal epochs for speed
# EPOCHS_CLS = 1           # ULTRA-FAST: Single epoch
# BATCH_SIZE = 128         # ULTRA-FAST: Large batches

# # Shared state
# LIVE_STATE = {}
# POLLERS = {}
# TRAIN_LOGS = {}       # summary logs stored after training finish
# TRAINING_FLAGS = {}   # symbol -> bool (training running)

# # ---------------- UTIL: safe model save/load ----------------
# def compile_model_for_saving(model):
#     # compile model with explicit objects to avoid short-name issues on load
#     model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#                   loss=tf.keras.losses.MeanSquaredError(),
#                   metrics=[tf.keras.metrics.MeanAbsoluteError()])
#     return model

# def safe_save_model(model, path):
#     """Save model in SavedModel format (directory) instead of H5"""
#     compile_model_for_saving(model)
#     # Use .keras format (TF 2.x native format) instead of legacy H5
#     keras_path = path.replace('.h5', '.keras')
#     model.save(keras_path, save_format='keras')
#     return keras_path

# def safe_load_model(path):
#     """Load model from .keras format with fallback to H5"""
#     keras_path = path.replace('.h5', '.keras')

#     # Try .keras format first
#     if os.path.exists(keras_path):
#         try:
#             return tf.keras.models.load_model(keras_path)
#         except Exception as e:
#             print(f"Failed to load .keras format: {e}")

#     # Fallback to H5 format
#     if os.path.exists(path):
#         try:
#             return tf.keras.models.load_model(path, compile=False)
#         except Exception as e:
#             print(f"Model load failed: {e}")
#             traceback.print_exc()

#     return None

# # ---------------- DATA & FEATURES ----------------
# def get_stock_history(symbol, period='4y', interval='1d'):
#     """
#     FIX #1: Properly handle yfinance DataFrame and ensure Close is 1D Series
#     FIX #3: Add auto_adjust parameter explicitly
#     """
#     # Download with explicit auto_adjust parameter to avoid warning
#     df = yf.download(symbol, period=period, interval=interval, progress=False, auto_adjust=True)

#     # Handle multi-level columns if present
#     if isinstance(df.columns, pd.MultiIndex):
#         df.columns = df.columns.get_level_values(0)

#     df = df[['Open','High','Low','Close','Volume']].copy()
#     df = df.dropna()

#     # CRITICAL FIX: Ensure Close column is 1D Series, not 2D DataFrame
#     if isinstance(df['Close'], pd.DataFrame):
#         df['Close'] = df['Close'].iloc[:, 0]

#     # Convert to Series if it's still not
#     close_series = pd.Series(df['Close'].values.flatten(), index=df.index)

#     # Technical indicators - pass Series explicitly
#     df['SMA_10'] = close_series.rolling(10).mean()
#     df['EMA_10'] = close_series.ewm(span=10, adjust=False).mean()

#     # RSI - ensure input is 1D
#     df['RSI_14'] = ta.momentum.rsi(close_series, window=14)

#     # MACD - ensure input is 1D
#     macd = ta.trend.MACD(close_series)
#     df['MACD'] = macd.macd()
#     df['MACD_signal'] = macd.macd_signal()

#     df.dropna(inplace=True)
#     return df

# def prepare_sequences(df, feature_cols=None, seq_len=SEQ_LEN):
#     if feature_cols is None:
#         feature_cols = ['Close','SMA_10','EMA_10','RSI_14','MACD','MACD_signal','Volume']

#     # Ensure all features are present
#     missing_cols = [col for col in feature_cols if col not in df.columns]
#     if missing_cols:
#         print(f"Warning: Missing columns {missing_cols}, using available columns only")
#         feature_cols = [col for col in feature_cols if col in df.columns]

#     arr = df[feature_cols].values.astype(np.float32)
#     scaler = MinMaxScaler()
#     scaled = scaler.fit_transform(arr)
#     X, y_reg, y_trend = [], [], []
#     for i in range(seq_len, len(scaled)):
#         X.append(scaled[i-seq_len:i])
#         y_reg.append(scaled[i][0])  # close at index 0
#         y_trend.append(1 if scaled[i][0] > scaled[i-1][0] else 0)
#     return np.array(X), np.array(y_reg), np.array(y_trend), scaler

# # ---------------- MODEL ARCHITECTURES ----------------
# def build_lstm(seq_len=SEQ_LEN, features=7):
#     """Ultra-fast LSTM - minimal architecture"""
#     inp = layers.Input(shape=(seq_len, features))
#     x = layers.LSTM(32)(inp)  # Single LSTM layer, reduced size
#     x = layers.Dense(16, activation='relu')(x)
#     out = layers.Dense(1)(x)
#     model = models.Model(inp, out)
#     model.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())
#     return model

# def build_transformer(seq_len=SEQ_LEN, features=7, d_model=16, heads=2, ff_dim=32):
#     """Ultra-fast Transformer - minimal architecture"""
#     inputs = layers.Input(shape=(seq_len, features))
#     x = layers.Dense(d_model)(inputs)
#     # Single attention block
#     attn = layers.MultiHeadAttention(num_heads=heads, key_dim=d_model)(x, x)
#     x = layers.Add()([x, attn])
#     x = layers.LayerNormalization()(x)
#     x = layers.GlobalAveragePooling1D()(x)
#     x = layers.Dense(16, activation='relu')(x)
#     outputs = layers.Dense(1)(x)
#     model = models.Model(inputs, outputs)
#     model.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())
#     return model

# def build_classifier(seq_len=SEQ_LEN, features=7):
#     """Ultra-fast Classifier - minimal architecture"""
#     inputs = layers.Input(shape=(seq_len, features))
#     x = layers.LSTM(32)(inputs)  # Single LSTM
#     x = layers.Dense(16, activation='relu')(x)
#     outputs = layers.Dense(1, activation='sigmoid')(x)
#     model = models.Model(inputs, outputs)
#     model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#     return model

# # ---------------- TRAINING (background thread) ----------------
# def train_models_thread(symbol, epochs_reg=EPOCHS_REG, epochs_cls=EPOCHS_CLS):
#     """
#     Train LSTM, Transformer, Classifier in a background thread.
#     Store final logs to TRAIN_LOGS[symbol].
#     """
#     TRAINING_FLAGS[symbol] = True
#     logs = []
#     def log(s):
#         ts = time.strftime("%H:%M:%S")
#         line = f"[{ts}] {s}"
#         logs.append(line)
#         TRAIN_LOGS[symbol] = "\n".join(logs)
#         print(line)

#     try:
#         log(f"Downloading {symbol} data...")
#         # ULTRA-FAST: Only 1 year of data
#         df = get_stock_history(symbol, period='1y', interval='1d')
#         log(f"✓ Downloaded {len(df)} rows")
#         X, y_reg, y_trend, scaler = prepare_sequences(df)
#         if len(X) < 50:  # Very low threshold
#             raise RuntimeError(f"Not enough training rows ({len(X)}). Need at least 50.")

#         split = int(len(X) * 0.8)
#         X_train, X_val = X[:split], X[split:]
#         y_train, y_val = y_reg[:split], y_reg[split:]
#         y_train_tr, y_val_tr = y_trend[:split], y_trend[split:]
#         log(f"✓ Prepared {len(X_train)} training samples, {len(X_val)} validation samples")

#         # LSTM
#         log("Building LSTM...")
#         lstm = build_lstm(seq_len=SEQ_LEN, features=X.shape[2])
#         log(f"Training LSTM ({epochs_reg} epochs)... ⏳")
#         history = lstm.fit(X_train, y_train, validation_data=(X_val,y_val),
#                           epochs=epochs_reg, batch_size=BATCH_SIZE, verbose=0)
#         log(f"✓ LSTM trained! Final loss: {history.history['loss'][-1]:.4f}")
#         lstm_path = os.path.join(MODELS_DIR, f"{symbol}_lstm.h5")
#         saved_path = safe_save_model(lstm, lstm_path)
#         log(f"✓ LSTM saved")

#         # Transformer
#         log("Building Transformer...")
#         trans = build_transformer(seq_len=SEQ_LEN, features=X.shape[2])
#         log(f"Training Transformer ({epochs_reg} epochs)... ⏳")
#         history = trans.fit(X_train, y_train, validation_data=(X_val,y_val),
#                            epochs=epochs_reg, batch_size=BATCH_SIZE, verbose=0)
#         log(f"✓ Transformer trained! Final loss: {history.history['loss'][-1]:.4f}")
#         trans_path = os.path.join(MODELS_DIR, f"{symbol}_trans.h5")
#         saved_path = safe_save_model(trans, trans_path)
#         log(f"✓ Transformer saved")

#         # Classifier
#         log("Building classifier...")
#         cls = build_classifier(seq_len=SEQ_LEN, features=X.shape[2])
#         log(f"Training Classifier ({epochs_cls} epoch)... ⏳")
#         history = cls.fit(X_train, y_train_tr, validation_data=(X_val, y_val_tr),
#                          epochs=epochs_cls, batch_size=BATCH_SIZE, verbose=0)
#         log(f"✓ Classifier trained! Accuracy: {history.history.get('val_accuracy', [0])[-1]:.2%}")
#         cls_path = os.path.join(MODELS_DIR, f"{symbol}_cls.h5")
#         saved_path = safe_save_model(cls, cls_path)
#         log(f"✓ Classifier saved")

#         # Scaler: save min and scale arrays (so we can inverse transform close)
#         scaler_path = os.path.join(MODELS_DIR, f"{symbol}_scaler.npy")
#         np.save(scaler_path, np.array([scaler.min_, scaler.scale_], dtype=object), allow_pickle=True)
#         log(f"✓ Scaler saved")

#         log("🎉 Training completed successfully!")
#         log(f"⏱️ Total time: ~30-60 seconds")
#     except Exception as e:
#         log(f"Training failed: {e}")
#         traceback.print_exc()
#     finally:
#         TRAINING_FLAGS[symbol] = False
#         TRAIN_LOGS[symbol] = "\n".join(logs)

# def start_training(symbol):
#     if TRAINING_FLAGS.get(symbol, False):
#         return "Training already running for " + symbol
#     t = threading.Thread(target=train_models_thread, args=(symbol,), daemon=True)
#     t.start()
#     return "Started background training for " + symbol

# # ---------------- helper: check trained and load models ----------------
# def is_trained(symbol):
#     """Check if all model files exist (both .keras and .h5 formats)"""
#     base_path = os.path.join(MODELS_DIR, symbol)

#     # Check for .keras format first (preferred)
#     keras_exists = (
#         os.path.exists(f"{base_path}_lstm.keras") and
#         os.path.exists(f"{base_path}_trans.keras") and
#         os.path.exists(f"{base_path}_cls.keras") and
#         os.path.exists(f"{base_path}_scaler.npy")
#     )

#     # Fallback to .h5 format
#     h5_exists = (
#         os.path.exists(f"{base_path}_lstm.h5") and
#         os.path.exists(f"{base_path}_trans.h5") and
#         os.path.exists(f"{base_path}_cls.h5") and
#         os.path.exists(f"{base_path}_scaler.npy")
#     )

#     return keras_exists or h5_exists

# def load_models(symbol):
#     """FIX #2: Improved model loading with better error handling"""
#     lstm = safe_load_model(os.path.join(MODELS_DIR, f"{symbol}_lstm.h5"))
#     trans = safe_load_model(os.path.join(MODELS_DIR, f"{symbol}_trans.h5"))
#     cls = safe_load_model(os.path.join(MODELS_DIR, f"{symbol}_cls.h5"))

#     if lstm is None or trans is None or cls is None:
#         raise RuntimeError(f"Failed to load one or more models for {symbol}")

#     # load scaler
#     arr = np.load(os.path.join(MODELS_DIR, f"{symbol}_scaler.npy"), allow_pickle=True)
#     min_, scale_ = arr

#     class ScalerWrap:
#         def __init__(self, min_, scale_):
#             self.min_ = np.array(min_, dtype=np.float32)
#             self.scale_ = np.array(scale_, dtype=np.float32)
#         def inv_close(self, scaled_close):
#             # scaled_close may be numpy array shape (n,1) or scalar
#             scaled_arr = np.atleast_1d(scaled_close)
#             return scaled_arr / self.scale_[0] + self.min_[0]

#     scl = ScalerWrap(min_, scale_)
#     return lstm, trans, cls, scl

# # ---------------- prediction helpers ----------------
# def ensemble_predict(lstm, trans, X):
#     p1 = lstm.predict(X, verbose=0).reshape(-1)
#     p2 = trans.predict(X, verbose=0).reshape(-1)
#     return (p1 + p2) / 2.0

# # ---------------- plotting helpers (always return matplotlib.figure) ----------------
# def plot_pred_vs_actual(actual, pred, title="Predicted vs Actual"):
#     fig = plt.figure(figsize=(10,4))
#     plt.plot(actual, label='Actual', linewidth=2)
#     plt.plot(pred, label='Predicted', linewidth=2, alpha=0.8)
#     plt.legend()
#     plt.title(title)
#     plt.xlabel('Time')
#     plt.ylabel('Price')
#     plt.grid(True, alpha=0.3)
#     plt.tight_layout()
#     return fig

# # ---------------- validation graph function ----------------
# def get_validation_figure(symbol):
#     if not is_trained(symbol):
#         # return a neutral figure urging training
#         fig = plt.figure(figsize=(8,3))
#         plt.text(0.5, 0.5, "Models not trained yet.\nClick 'Train (background)' button.",
#                 ha='center', va='center', fontsize=14)
#         plt.axis('off')
#         return fig

#     try:
#         lstm, trans, cls, scl = load_models(symbol)
#         df = get_stock_history(symbol, period='3y', interval='1d')
#         X, y_reg, y_trend, scaler = prepare_sequences(df)
#         if len(X) == 0:
#             fig = plt.figure(figsize=(8,3))
#             plt.text(0.5,0.5,"Not enough data for validation plot.", ha='center', va='center')
#             plt.axis('off')
#             return fig

#         split = int(len(X) * 0.8)
#         X_val = X[split:]
#         y_val = y_reg[split:]
#         scaled_preds = ensemble_predict(lstm, trans, X_val)

#         # Inverse transform
#         preds = scaler.inverse_transform(scaled_preds.reshape(-1,1)).reshape(-1)
#         actuals = scaler.inverse_transform(y_val.reshape(-1,1)).reshape(-1)

#         return plot_pred_vs_actual(actuals, preds, title=f"{symbol} Validation")
#     except Exception as e:
#         fig = plt.figure(figsize=(8,3))
#         plt.text(0.5, 0.5, f"Error generating plot:\n{str(e)}",
#                 ha='center', va='center', fontsize=10)
#         plt.axis('off')
#         return fig

# # ---------------- live poller (yfinance) ----------------
# def poller(symbol, stop_event):
#     # ensure model trained
#     if not is_trained(symbol):
#         # train synchronously to avoid poller running with no model
#         train_models_thread(symbol)
#         # after training returns, continue (or abort if missing)

#     try:
#         lstm, trans, cls, scl = load_models(symbol)
#     except Exception as e:
#         print(f"Poller failed to load models: {e}")
#         LIVE_STATE.pop(symbol, None)
#         return

#     while not stop_event.is_set():
#         try:
#             # try 1m if available, else daily
#             df = yf.download(symbol, period='5d', interval='1m', progress=False, auto_adjust=True)
#             if df.empty:
#                 df = get_stock_history(symbol, period='6mo', interval='1d')
#             else:
#                 # Handle multi-level columns
#                 if isinstance(df.columns, pd.MultiIndex):
#                     df.columns = df.columns.get_level_values(0)
#                 df = df[['Open','High','Low','Close','Volume']].dropna()

#             # prepare feature df
#             df_full = get_stock_history(symbol, period='6mo', interval='1d')
#             X, y_reg, y_trend, scaler = prepare_sequences(df_full)
#             if len(X) < 1:
#                 time.sleep(POLL_INTERVAL)
#                 continue

#             last = X[-1].reshape(1, X.shape[1], X.shape[2])
#             scaled_pred = ensemble_predict(lstm, trans, last)[0]
#             pred_price = scl.inv_close(np.array([[scaled_pred]]))[0]

#             LIVE_STATE[symbol] = {
#                 'timestamp': time.time(),
#                 'pred_price': float(pred_price),
#                 'actual_price': float(df['Close'].values[-1]),
#                 'history_df': df_full.tail(300)
#             }
#         except Exception as e:
#             print(f"Poller error for {symbol}: {e}")

#         time.sleep(POLL_INTERVAL)

# def start_poller(symbol):
#     if symbol in POLLERS:
#         return f"Poller already running for {symbol}"
#     if not is_trained(symbol):
#         return f"Models not trained for {symbol}. Train first!"

#     stop_event = threading.Event()
#     t = threading.Thread(target=poller, args=(symbol, stop_event), daemon=True)
#     POLLERS[symbol] = {'thread': t, 'stop': stop_event}
#     t.start()
#     return f"Started live poller for {symbol}"

# def stop_poller(symbol):
#     if symbol not in POLLERS:
#         return f"No poller running for {symbol}"
#     POLLERS[symbol]['stop'].set()
#     time.sleep(0.5)
#     del POLLERS[symbol]
#     return f"Stopped poller for {symbol}"

# def get_live_figure(symbol):
#     s = LIVE_STATE.get(symbol)
#     if not s:
#         fig = plt.figure(figsize=(8,3))
#         plt.text(0.5,0.5, "No live data yet.\nStart Poller first.", ha='center', va='center', fontsize=12)
#         plt.axis('off')
#         return fig

#     df = s.get('history_df')
#     if df is None or df.empty:
#         fig = plt.figure(figsize=(8,3))
#         plt.text(0.5,0.5,"No history available", ha='center', va='center')
#         plt.axis('off')
#         return fig

#     close = df['Close'].values
#     fig = plt.figure(figsize=(10,4))
#     plt.plot(close, label='Recent Close', linewidth=2)
#     plt.scatter([len(close)-1], [s['pred_price']], color='red', s=100,
#                label=f"Predicted: ${s['pred_price']:.2f}", zorder=5)
#     plt.legend()
#     plt.title(f"{symbol} Live (recent closes + current prediction)")
#     plt.xlabel('Time')
#     plt.ylabel('Price ($)')
#     plt.grid(True, alpha=0.3)
#     plt.tight_layout()
#     return fig

# # ---------------- single prediction ----------------
# def single_prediction(symbol):
#     if not is_trained(symbol):
#         return f"⚠️ Models not trained for {symbol}. Click 'Train (background)' first."

#     try:
#         lstm, trans, cls, scl = load_models(symbol)
#         df = get_stock_history(symbol, period='6mo', interval='1d')
#         X, y_reg, y_trend, scaler = prepare_sequences(df)
#         if len(X) == 0:
#             return "Not enough data to predict."

#         last = X[-1].reshape(1, X.shape[1], X.shape[2])
#         scaled_pred = ensemble_predict(lstm, trans, last)[0]
#         pred_price = scl.inv_close(np.array([[scaled_pred]]))[0]

#         # Get current price
#         current_price = df['Close'].values[-1]
#         change = ((pred_price - current_price) / current_price) * 100

#         return (f"📊 Prediction for {symbol}\n"
#                f"Current Price: ${current_price:.2f}\n"
#                f"Predicted Next Close: ${pred_price:.2f}\n"
#                f"Expected Change: {change:+.2f}%")
#     except Exception as e:
#         return f"❌ Error making prediction: {str(e)}"

# # ---------------- model download helper ----------------
# def download_paths(symbol):
#     if not is_trained(symbol):
#         return (None, None, None, None)

#     # Check for .keras files first
#     base = os.path.join(MODELS_DIR, symbol)
#     lstm_path = f"{base}_lstm.keras" if os.path.exists(f"{base}_lstm.keras") else f"{base}_lstm.h5"
#     trans_path = f"{base}_trans.keras" if os.path.exists(f"{base}_trans.keras") else f"{base}_trans.h5"
#     cls_path = f"{base}_cls.keras" if os.path.exists(f"{base}_cls.keras") else f"{base}_cls.h5"
#     scaler_path = f"{base}_scaler.npy"

#     return (lstm_path, trans_path, cls_path, scaler_path)

# # ---------------- GRADIO UI ----------------
# with gr.Blocks(theme=gr.themes.Soft()) as demo:
#     gr.Markdown("# 📈 Top-10 Stock Predictor — Ensemble LSTM+Transformer")
#     gr.Markdown("Train models, make predictions, and monitor live stock prices with AI")

#     with gr.Row():
#         stock_sel = gr.Dropdown(TOP_10_STOCKS, label="Select Stock Symbol", value="AAPL")
#         train_btn = gr.Button("🚀 Train (background)", variant="primary")

#     with gr.Row():
#         logs_box = gr.Textbox(label="Training Logs", lines=10, max_lines=15)
#         logs_refresh = gr.Button("🔄 Refresh Logs")

#     gr.Markdown("---")
#     gr.Markdown("## 📊 Predictions")

#     with gr.Row():
#         predict_btn = gr.Button("🎯 Single Prediction", variant="secondary")
#         pred_box = gr.Textbox(label="Prediction Result", lines=4)

#     with gr.Row():
#         val_plot = gr.Plot(label="Validation: Predicted vs Actual")

#     gr.Markdown("---")
#     gr.Markdown("## 📡 Live Monitoring")

#     with gr.Row():
#         start_btn = gr.Button("▶️ Start Live Polling", variant="primary")
#         stop_btn = gr.Button("⏸️ Stop Polling", variant="stop")
#         live_refresh = gr.Button("🔄 Refresh Live Data")

#     with gr.Row():
#         live_status = gr.Textbox(label="Live Status", lines=3)
#         live_plot = gr.Plot(label="Live Price Chart")

#     gr.Markdown("---")
#     gr.Markdown("## 💾 Download Trained Models")

#     with gr.Row():
#         dl_lstm = gr.File(label="LSTM Model")
#         dl_trans = gr.File(label="Transformer Model")
#         dl_cls = gr.File(label="Classifier Model")
#         dl_scaler = gr.File(label="Scaler Data")

#     # Wire up all the interactions
#     train_btn.click(fn=start_training, inputs=stock_sel, outputs=logs_box)

#     def fetch_logs(symbol):
#         return TRAIN_LOGS.get(symbol, "No logs yet. Click 'Train (background)' to start.")

#     logs_refresh.click(fn=fetch_logs, inputs=stock_sel, outputs=logs_box)
#     stock_sel.change(fn=get_validation_figure, inputs=stock_sel, outputs=val_plot)
#     predict_btn.click(fn=single_prediction, inputs=stock_sel, outputs=pred_box)

#     start_btn.click(fn=start_poller, inputs=stock_sel, outputs=live_status)
#     stop_btn.click(fn=stop_poller, inputs=stock_sel, outputs=live_status)

#     def live_status_and_plot(symbol):
#         s = LIVE_STATE.get(symbol)
#         if not s:
#             return f"No live data for {symbol}. Start poller first.", get_live_figure(symbol)
#         text = (f"📍 {symbol} Live Data\n"
#                 f"Last Update: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(s['timestamp']))}\n"
#                 f"Current Price: ${s['actual_price']:.2f} | Predicted: ${s['pred_price']:.2f}")
#         return text, get_live_figure(symbol)

#     live_refresh.click(fn=live_status_and_plot, inputs=stock_sel, outputs=[live_status, live_plot])
#     stock_sel.change(fn=download_paths, inputs=stock_sel, outputs=[dl_lstm, dl_trans, dl_cls, dl_scaler])

# # Launch Gradio app
# if __name__ == "__main__":
#     print("🚀 Starting Stock Predictor App...")
#     demo.launch(share=True, debug=False)

🚀 Starting Stock Predictor App...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9bd58ca319ddd73a1b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [4]:
pip install yfinance gradio joblib tensorflow scikit-learn matplotlib

In [5]:
# # FULLY DEBUGGED Stock Predictor - LSTM + Bidirectional LSTM Only
# # Copy into Colab / local env and run. Requires: yfinance, tensorflow, scikit-learn, joblib, gradio, matplotlib, pandas, numpy

# import os, time, threading, traceback, warnings, shutil
# import numpy as np
# import pandas as pd
# import yfinance as yf
# from sklearn.preprocessing import MinMaxScaler
# import joblib
# import tensorflow as tf
# from tensorflow.keras import layers, models, callbacks
# import matplotlib
# matplotlib.use('Agg')  # Non-interactive backend
# import matplotlib.pyplot as plt
# import gradio as gr

# # Suppress warnings
# warnings.filterwarnings('ignore')
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# # ---------------- CONFIG ----------------
# TOP_10_STOCKS = ["AAPL","MSFT","AMZN","GOOGL","META","TSLA","NVDA","NFLX","JPM","AMD"]
# MODELS_DIR = "/content/stock_models"
# os.makedirs(MODELS_DIR, exist_ok=True)

# # OPTIMIZED FOR SPEED (adjust for real training)
# SEQ_LEN = 20
# POLL_INTERVAL = 10
# EPOCHS = 3
# BATCH_SIZE = 64

# # Shared state
# LIVE_STATE = {}
# POLLERS = {}
# TRAIN_LOGS = {}
# TRAINING_FLAGS = {}

# # ---------------- LOGGING UTILITY ----------------
# class Logger:
#     def __init__(self, symbol):
#         self.symbol = symbol
#         self.logs = TRAIN_LOGS.get(symbol, []).copy() if isinstance(TRAIN_LOGS.get(symbol, []), list) else []
#     def log(self, msg, level="INFO"):
#         timestamp = time.strftime("%H:%M:%S")
#         emoji = {"INFO": "ℹ️", "SUCCESS": "✅", "ERROR": "❌", "WARNING": "⚠️"}.get(level, "📝")
#         line = f"[{timestamp}] {emoji} {msg}"
#         self.logs.append(line)
#         TRAIN_LOGS[self.symbol] = self.logs
#         # also return a human string
#         print(line)
#         return line

# def logs_to_text(symbol):
#     logs = TRAIN_LOGS.get(symbol, [])
#     if not logs:
#         return "No logs yet. Click 'Train Models' to start."
#     status = "🔄 Training in progress...\n\n" if TRAINING_FLAGS.get(symbol, False) else "✅ Training completed (or not started)\n\n"
#     return status + "\n".join(logs)

# # ---------------- DATA FUNCTIONS ----------------
# def download_stock_data(symbol, period='1y', logger=None):
#     try:
#         if logger:
#             logger.log(f"Downloading {symbol} data ({period})...")
#         df = yf.download(symbol, period=period, interval='1d', progress=False, auto_adjust=True, threads=False)
#         if df is None or df.empty:
#             raise ValueError(f"No data returned for {symbol} (period={period})")
#         # if multiindex columns
#         if isinstance(df.columns, pd.MultiIndex):
#             df.columns = df.columns.get_level_values(0)
#         required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
#         missing = [c for c in required_cols if c not in df.columns]
#         if missing:
#             raise ValueError(f"Missing required columns from Yahoo: {missing}")
#         df = df[required_cols].copy()
#         df = df.dropna()
#         if logger:
#             logger.log(f"Downloaded {len(df)} rows", "SUCCESS")
#         return df
#     except Exception as e:
#         if logger:
#             logger.log(f"Download failed: {str(e)}", "ERROR")
#         raise

# def add_technical_indicators(df, logger=None):
#     try:
#         if logger:
#             logger.log("Adding technical indicators...")
#         df = df.copy()
#         close = df['Close']
#         df['SMA_10'] = close.rolling(window=10, min_periods=1).mean()
#         df['EMA_10'] = close.ewm(span=10, adjust=False).mean()
#         delta = close.diff()
#         gain = delta.where(delta > 0, 0.0)
#         loss = -delta.where(delta < 0, 0.0)
#         avg_gain = gain.rolling(window=14, min_periods=1).mean()
#         avg_loss = loss.rolling(window=14, min_periods=1).mean()
#         rs = avg_gain / (avg_loss + 1e-8)
#         df['RSI_14'] = 100 - (100 / (1 + rs))
#         df['Volume_Change'] = df['Volume'].pct_change().fillna(0.0)
#         df = df.dropna()
#         if logger:
#             logger.log(f"Added indicators, {len(df)} rows remaining", "SUCCESS")
#         return df
#     except Exception as e:
#         if logger:
#             logger.log(f"Indicator calculation failed: {str(e)}", "ERROR")
#         raise

# def prepare_sequences(df, seq_len=SEQ_LEN, logger=None):
#     try:
#         if logger:
#             logger.log("Preparing sequences...")
#         feature_cols = ['Close', 'SMA_10', 'EMA_10', 'RSI_14', 'Volume_Change']
#         missing = [col for col in feature_cols if col not in df.columns]
#         if missing:
#             raise ValueError(f"Missing columns for sequence prep: {missing}")
#         arr = df[feature_cols].values.astype(np.float32)
#         scaler = MinMaxScaler()
#         scaled = scaler.fit_transform(arr)
#         X, y = [], []
#         for i in range(seq_len, len(scaled)):
#             X.append(scaled[i-seq_len:i])
#             y.append(scaled[i, 0])  # predict the scaled Close value
#         X = np.array(X, dtype=np.float32)
#         y = np.array(y, dtype=np.float32)
#         if logger:
#             logger.log(f"Created {len(X)} sequences", "SUCCESS")
#         n_features = arr.shape[1]
#         return X, y, scaler, n_features
#     except Exception as e:
#         if logger:
#             logger.log(f"Sequence preparation failed: {str(e)}", "ERROR")
#         raise

# # ---------------- MODEL ARCHITECTURES ----------------
# def build_lstm_model(seq_len, n_features):
#     model = models.Sequential([
#         layers.Input(shape=(seq_len, n_features)),
#         layers.LSTM(64, return_sequences=True),
#         layers.Dropout(0.2),
#         layers.LSTM(32),
#         layers.Dense(16, activation='relu'),
#         layers.Dense(1)
#     ])
#     model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse', metrics=['mae'])
#     return model

# def build_bidirectional_lstm_model(seq_len, n_features):
#     model = models.Sequential([
#         layers.Input(shape=(seq_len, n_features)),
#         layers.Bidirectional(layers.LSTM(64, return_sequences=True)),
#         layers.Dropout(0.2),
#         layers.Bidirectional(layers.LSTM(32)),
#         layers.Dense(16, activation='relu'),
#         layers.Dense(1)
#     ])
#     model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse', metrics=['mae'])
#     return model

# # ---------------- MODEL SAVE/LOAD (robust) ----------------
# def save_model_tf(model, save_dir, logger):
#     try:
#         os.makedirs(save_dir, exist_ok=True)
#         save_path = os.path.join(save_dir, "model.keras")

#         # Use .log() instead of calling logger(...)
#         logger.log(f"ℹ️ Saving model to: {save_path}")

#         model.save(save_path)

#         logger.log("✅ Model saved successfully!")

#     except Exception as e:
#         logger.log(f"❌ Save failed: {e}")



# def load_model_tf(save_dir, logger):
#     try:
#         model_path = os.path.join(save_dir, "model.keras")

#         if not os.path.exists(model_path):
#             logger.log("❌ No saved model found.")
#             return None

#         logger.log(f"ℹ️ Loading model from: {model_path}")

#         model = tf.keras.models.load_model(model_path)

#         logger.log("✅ Model loaded successfully!")
#         return model

#     except Exception as e:
#         logger.log(f"❌ Load failed: {e}")
#         return None


# # ---------------- TRAINING THREAD ----------------
# def train_models_thread(symbol):
#     TRAINING_FLAGS[symbol] = True
#     logger = Logger(symbol)
#     TRAIN_LOGS[symbol] = TRAIN_LOGS.get(symbol, [])  # ensure exists
#     try:
#         logger.log(f"🚀 Starting training for {symbol}")
#         df = download_stock_data(symbol, period='1y', logger=logger)
#         df = add_technical_indicators(df, logger=logger)
#         X, y, scaler, n_features = prepare_sequences(df, SEQ_LEN, logger=logger)
#         if len(X) < 100:
#             raise ValueError(f"Insufficient data: {len(X)} sequences (need 100+ for reliable training)")
#         split_idx = int(len(X) * 0.8)
#         X_train, X_test = X[:split_idx], X[split_idx:]
#         y_train, y_test = y[:split_idx], y[split_idx:]
#         logger.log(f"Training: {len(X_train)} samples, Testing: {len(X_test)} samples")
#         # LSTM
#         logger.log("🏗️ Building LSTM model...")
#         lstm_model = build_lstm_model(SEQ_LEN, n_features)
#         logger.log(f"🎓 Training LSTM ({EPOCHS} epochs)...")
#         history_lstm = lstm_model.fit(
#             X_train, y_train,
#             validation_data=(X_test, y_test),
#             epochs=EPOCHS,
#             batch_size=BATCH_SIZE,
#             verbose=0,
#             callbacks=[callbacks.EarlyStopping(patience=2, restore_best_weights=True)]
#         )
#         final_loss = history_lstm.history.get('val_loss', [None])[-1]
#         logger.log(f"LSTM trained! Val Loss: {final_loss:.6f}", "SUCCESS")
#         lstm_dir = os.path.join(MODELS_DIR, f"{symbol}_lstm_saved_model")
#         save_model_tf(lstm_model, lstm_dir, logger)
#         # Bidirectional LSTM
#         logger.log("🏗️ Building Bidirectional LSTM model...")
#         bilstm_model = build_bidirectional_lstm_model(SEQ_LEN, n_features)
#         logger.log(f"🎓 Training Bidirectional LSTM ({EPOCHS} epochs)...")
#         history_bilstm = bilstm_model.fit(
#             X_train, y_train,
#             validation_data=(X_test, y_test),
#             epochs=EPOCHS,
#             batch_size=BATCH_SIZE,
#             verbose=0,
#             callbacks=[callbacks.EarlyStopping(patience=2, restore_best_weights=True)]
#         )
#         final_loss2 = history_bilstm.history.get('val_loss', [None])[-1]
#         logger.log(f"Bidirectional LSTM trained! Val Loss: {final_loss2:.6f}", "SUCCESS")
#         bilstm_dir = os.path.join(MODELS_DIR, f"{symbol}_bilstm_saved_model")
#         save_model_tf(bilstm_model, bilstm_dir, logger)
#         # Save scaler with joblib for reliable inverse_transform
#         scaler_path = os.path.join(MODELS_DIR, f"{symbol}_scaler.joblib")
#         joblib.dump(scaler, scaler_path)
#         logger.log(f"Scaler saved to {scaler_path}", "SUCCESS")
#         # Optionally zip the saved model directories for download
#         for d in (lstm_dir, bilstm_dir):
#             base = d  # full dir path
#             zip_path = f"{base}.zip"
#             if os.path.exists(zip_path):
#                 os.remove(zip_path)
#             shutil.make_archive(base, 'zip', base)
#             logger.log(f"Created archive {zip_path}", "INFO")
#         logger.log("🎉 Training completed successfully!")
#     except Exception as e:
#         logger.log(f"Training failed: {str(e)}", "ERROR")
#         tb = traceback.format_exc()
#         for line in tb.splitlines():
#             logger.log(line, "ERROR")
#     finally:
#         TRAINING_FLAGS[symbol] = False

# # ---------------- HELPERS ----------------
# def is_trained(symbol):
#     lstm_dir = os.path.join(MODELS_DIR, f"{symbol}_lstm_saved_model")
#     bilstm_dir = os.path.join(MODELS_DIR, f"{symbol}_bilstm_saved_model")
#     scaler_path = os.path.join(MODELS_DIR, f"{symbol}_scaler.joblib")
#     return os.path.exists(lstm_dir) and os.path.exists(bilstm_dir) and os.path.exists(scaler_path)

# def load_models_and_scaler(symbol):
#     logger = Logger(symbol)
#     lstm_dir = os.path.join(MODELS_DIR, f"{symbol}_lstm_saved_model")
#     bilstm_dir = os.path.join(MODELS_DIR, f"{symbol}_bilstm_saved_model")
#     scaler_path = os.path.join(MODELS_DIR, f"{symbol}_scaler.joblib")
#     lstm_model = load_model_tf(lstm_dir, logger)
#     bilstm_model = load_model_tf(bilstm_dir, logger)
#     if lstm_model is None or bilstm_model is None:
#         raise RuntimeError("Failed to load models")
#     if not os.path.exists(scaler_path):
#         raise FileNotFoundError("Scaler file not found")
#     scaler = joblib.load(scaler_path)
#     return lstm_model, bilstm_model, scaler

# def ensemble_predict(lstm_model, bilstm_model, X):
#     p1 = lstm_model.predict(X, verbose=0).flatten()
#     p2 = bilstm_model.predict(X, verbose=0).flatten()
#     return (p1 + p2) / 2.0

# # ---------------- UI FUNCTIONS ----------------
# def start_training(symbol):
#     if TRAINING_FLAGS.get(symbol, False):
#         return f"⚠️ Training already running for {symbol}"
#     # clear logs for symbol
#     TRAIN_LOGS[symbol] = []
#     thread = threading.Thread(target=train_models_thread, args=(symbol,), daemon=True)
#     thread.start()
#     return f"🚀 Started training for {symbol}. Click 'Refresh Logs' to view progress."

# def get_training_logs(symbol):
#     return logs_to_text(symbol)

# def make_prediction(symbol):
#     try:
#         if not is_trained(symbol):
#             return f"❌ Models not trained for {symbol}. Train first!"
#         lstm_model, bilstm_model, scaler = load_models_and_scaler(symbol)
#         df = download_stock_data(symbol, period='3mo')
#         df = add_technical_indicators(df)
#         X, y, _, _ = prepare_sequences(df, SEQ_LEN)
#         if len(X) == 0:
#             return "❌ Insufficient data for prediction"
#         last_seq = X[-1:]
#         pred_scaled = ensemble_predict(lstm_model, bilstm_model, last_seq)[0]
#         # Create dummy vector for inverse transform (same n_features)
#         n_features = scaler.scale_.shape[0]
#         dummy = np.zeros((1, n_features), dtype=np.float32)
#         dummy[0, 0] = pred_scaled
#         pred_price = scaler.inverse_transform(dummy)[0, 0]
#         current_price = df['Close'].values[-1]
#         change_pct = ((pred_price - current_price) / current_price) * 100
#         result = (
#             f"📊 Prediction for {symbol}\n\n"
#             f"Current Price: ${current_price:.2f}\n"
#             f"Predicted Next: ${pred_price:.2f}\n"
#             f"Expected Change: {change_pct:+.2f}%\n\n"
#             f"🤖 Ensemble: LSTM + Bidirectional LSTM"
#         )
#         return result
#     except Exception as e:
#         return f"❌ Error during prediction: {str(e)}"

# def get_validation_plot(symbol):
#     try:
#         if not is_trained(symbol):
#             fig = plt.figure(figsize=(10, 4))
#             plt.text(0.5, 0.5, f"Models not trained for {symbol}\nClick 'Train Models' first", ha='center', va='center', fontsize=14)
#             plt.axis('off')
#             return fig
#         lstm_model, bilstm_model, scaler = load_models_and_scaler(symbol)
#         df = download_stock_data(symbol, period='6mo')
#         df = add_technical_indicators(df)
#         X, y, _, _ = prepare_sequences(df, SEQ_LEN)
#         if len(X) < 2:
#             fig = plt.figure(figsize=(10, 4))
#             plt.text(0.5, 0.5, "Insufficient data for validation plot", ha='center', va='center', fontsize=12)
#             plt.axis('off')
#             return fig
#         split_idx = int(len(X) * 0.8)
#         X_test = X[split_idx:]
#         y_test = y[split_idx:]
#         pred_scaled = ensemble_predict(lstm_model, bilstm_model, X_test)
#         # inverse transform
#         n_features = scaler.scale_.shape[0]
#         dummy_pred = np.zeros((len(pred_scaled), n_features), dtype=np.float32)
#         dummy_pred[:, 0] = pred_scaled
#         pred_prices = scaler.inverse_transform(dummy_pred)[:, 0]
#         dummy_actual = np.zeros((len(y_test), n_features), dtype=np.float32)
#         dummy_actual[:, 0] = y_test
#         actual_prices = scaler.inverse_transform(dummy_actual)[:, 0]
#         fig = plt.figure(figsize=(12, 5))
#         plt.plot(actual_prices, label='Actual', linewidth=2)
#         plt.plot(pred_prices, label='Predicted', linewidth=2, alpha=0.7)
#         plt.title(f'{symbol} - Validation: Actual vs Predicted', fontsize=14, fontweight='bold')
#         plt.xlabel('Time Steps')
#         plt.ylabel('Price ($)')
#         plt.legend()
#         plt.grid(True, alpha=0.3)
#         plt.tight_layout()
#         return fig
#     except Exception as e:
#         fig = plt.figure(figsize=(10, 4))
#         plt.text(0.5, 0.5, f"Error generating plot:\n{str(e)}", ha='center', va='center', fontsize=12)
#         plt.axis('off')
#         return fig

# def download_model_files(symbol):
#     if not is_trained(symbol):
#         return None, None, None
#     lstm_dir = os.path.join(MODELS_DIR, f"{symbol}_lstm_saved_model")
#     bilstm_dir = os.path.join(MODELS_DIR, f"{symbol}_bilstm_saved_model")
#     scaler_path = os.path.join(MODELS_DIR, f"{symbol}_scaler.joblib")
#     lstm_zip = f"{lstm_dir}.zip"
#     bilstm_zip = f"{bilstm_dir}.zip"
#     # create zips if missing (safe)
#     if not os.path.exists(lstm_zip):
#         shutil.make_archive(lstm_dir, 'zip', lstm_dir)
#     if not os.path.exists(bilstm_zip):
#         shutil.make_archive(bilstm_dir, 'zip', bilstm_dir)
#     return lstm_zip, bilstm_zip, scaler_path

# # ---------------- GRADIO INTERFACE ----------------
# with gr.Blocks(theme=gr.themes.Soft(), title="Stock Predictor") as app:
#     gr.Markdown("""
#     # 📈 Stock Price Predictor
#     ### LSTM + Bidirectional LSTM Ensemble Model
#     Train models on historical data and predict future stock prices
#     """)
#     with gr.Row():
#         stock_dropdown = gr.Dropdown(choices=TOP_10_STOCKS, value="AAPL", label="📊 Select Stock", interactive=True)
#         train_button = gr.Button("🚀 Train Models", variant="primary", size="lg")
#     with gr.Row():
#         logs_display = gr.Textbox(label="📋 Training Logs", lines=12, max_lines=20, interactive=False)
#         refresh_logs_btn = gr.Button("🔄 Refresh Logs")
#     gr.Markdown("---")
#     gr.Markdown("## 🎯 Make Predictions")
#     with gr.Row():
#         predict_button = gr.Button("🔮 Predict Next Price", variant="secondary", size="lg")
#         prediction_output = gr.Textbox(label="Prediction Result", lines=7, interactive=False)
#     with gr.Row():
#         validation_plot = gr.Plot(label="📊 Validation Plot (Actual vs Predicted)")
#     gr.Markdown("---")
#     gr.Markdown("## 💾 Download Trained Models")
#     with gr.Row():
#         lstm_download = gr.File(label="LSTM Model (ZIP)")
#         bilstm_download = gr.File(label="Bidirectional LSTM Model (ZIP)")
#         scaler_download = gr.File(label="Scaler Data")
#     # Events
#     train_button.click(fn=start_training, inputs=[stock_dropdown], outputs=[logs_display])
#     refresh_logs_btn.click(fn=get_training_logs, inputs=[stock_dropdown], outputs=[logs_display])
#     predict_button.click(fn=make_prediction, inputs=[stock_dropdown], outputs=[prediction_output])
#     stock_dropdown.change(fn=get_validation_plot, inputs=[stock_dropdown], outputs=[validation_plot])
#     stock_dropdown.change(fn=download_model_files, inputs=[stock_dropdown], outputs=[lstm_download, bilstm_download, scaler_download])

# if __name__ == "__main__":
#     print("=" * 50)
#     print("🚀 Stock Predictor App Starting...")
#     print("=" * 50)
#     app.launch(share=True, debug=False)


🚀 Stock Predictor App Starting...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2eab905bf587dbb501.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [10]:
import os, time, threading, traceback, warnings, shutil
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
import joblib
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import matplotlib
matplotlib.use('Agg')  # Non-interactive backend for Colab/Gradio
import matplotlib.pyplot as plt
import gradio as gr

# Suppress noisy warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# ---------------- CONFIG ----------------
TOP_10_STOCKS = ["AAPL","MSFT","AMZN","GOOGL","META","TSLA","NVDA","NFLX","JPM","AMD"]
MODELS_DIR = "/content/stock_models"
os.makedirs(MODELS_DIR, exist_ok=True)

# Quick test defaults (increase for real training)
SEQ_LEN = 20
EPOCHS = 30
BATCH_SIZE = 64

# Shared state
TRAIN_LOGS = {}
TRAINING_FLAGS = {}

# ---------------- LOGGING ----------------
class Logger:
    def __init__(self, symbol):
        self.symbol = symbol
        # ensure logs list exists
        TRAIN_LOGS.setdefault(symbol, [])
        self.logs = TRAIN_LOGS[symbol]

    def log(self, msg, level="INFO"):
        timestamp = time.strftime("%H:%M:%S")
        emoji = {"INFO": "ℹ️", "SUCCESS": "✅", "ERROR": "❌", "WARNING": "⚠️"}.get(level, "📝")
        line = f"[{timestamp}] {emoji} {msg}"
        self.logs.append(line)
        TRAIN_LOGS[self.symbol] = self.logs
        print(line)
        return line

def logs_to_text(symbol):
    logs = TRAIN_LOGS.get(symbol, [])
    if not logs:
        return "No logs yet. Click 'Train Models' to start."
    status = "🔄 Training in progress...\n\n" if TRAINING_FLAGS.get(symbol, False) else "✅ Training completed (or not started)\n\n"
    return status + "\n".join(logs)

# ---------------- DATA UTILITIES ----------------
def download_stock_data(symbol, period='1y', logger=None):
    try:
        if logger:
            logger.log(f"Downloading {symbol} data ({period})...")
        df = yf.download(symbol, period=period, interval='1d', progress=False, auto_adjust=True, threads=False)
        if df is None or df.empty:
            raise ValueError(f"No data returned for {symbol} (period={period})")
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
        required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
        missing = [c for c in required_cols if c not in df.columns]
        if missing:
            raise ValueError(f"Missing required columns from Yahoo: {missing}")
        df = df[required_cols].copy()
        df = df.dropna()
        if logger:
            logger.log(f"Downloaded {len(df)} rows", "SUCCESS")
        return df
    except Exception as e:
        if logger:
            logger.log(f"Download failed: {str(e)}", "ERROR")
        raise

def add_technical_indicators(df, logger=None):
    try:
        if logger:
            logger.log("Adding technical indicators...")
        df = df.copy()
        close = df['Close']
        df['SMA_10'] = close.rolling(window=10, min_periods=1).mean()
        df['EMA_10'] = close.ewm(span=10, adjust=False).mean()
        delta = close.diff()
        gain = delta.where(delta > 0, 0.0)
        loss = -delta.where(delta < 0, 0.0)
        avg_gain = gain.rolling(window=14, min_periods=1).mean()
        avg_loss = loss.rolling(window=14, min_periods=1).mean()
        rs = avg_gain / (avg_loss + 1e-8)
        df['RSI_14'] = 100 - (100 / (1 + rs))
        df['Volume_Change'] = df['Volume'].pct_change().fillna(0.0)
        df = df.dropna()
        if logger:
            logger.log(f"Added indicators, {len(df)} rows remaining", "SUCCESS")
        return df
    except Exception as e:
        if logger:
            logger.log(f"Indicator calculation failed: {str(e)}", "ERROR")
        raise

def prepare_sequences(df, seq_len=SEQ_LEN, logger=None):
    try:
        if logger:
            logger.log("Preparing sequences...")
        feature_cols = ['Close', 'SMA_10', 'EMA_10', 'RSI_14', 'Volume_Change']
        missing = [col for col in feature_cols if col not in df.columns]
        if missing:
            raise ValueError(f"Missing columns for sequence prep: {missing}")
        arr = df[feature_cols].values.astype(np.float32)
        scaler = MinMaxScaler()
        scaled = scaler.fit_transform(arr)
        X, y = [], []
        for i in range(seq_len, len(scaled)):
            X.append(scaled[i-seq_len:i])
            y.append(scaled[i, 0])
        X = np.array(X, dtype=np.float32)
        y = np.array(y, dtype=np.float32)
        if logger:
            logger.log(f"Created {len(X)} sequences", "SUCCESS")
        n_features = arr.shape[1]
        return X, y, scaler, n_features
    except Exception as e:
        if logger:
            logger.log(f"Sequence preparation failed: {str(e)}", "ERROR")
        raise

# ---------------- MODELS ----------------
def build_lstm_model(seq_len, n_features):
    model = models.Sequential([
        layers.Input(shape=(seq_len, n_features)),
        layers.LSTM(128, return_sequences=True),
        layers.Dropout(0.2),
        layers.LSTM(64),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss='mse', metrics=['mae'])
    return model

def build_bidirectional_lstm_model(seq_len, n_features):
    model = models.Sequential([
        layers.Input(shape=(seq_len, n_features)),
        layers.Bidirectional(layers.LSTM(64, return_sequences=True)),
        layers.Dropout(0.2),
        layers.Bidirectional(layers.LSTM(64)),
        layers.Dense(16, activation='relu'),
        layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss='mse', metrics=['mae'])
    return model

# ---------------- SAVE / LOAD (Keras 3 friendly) ----------------
def save_model_tf(model, save_dir, logger):
    try:
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, "model.keras")
        logger.log(f"ℹ️ Saving model to: {save_path}")
        # Keras 3 will use .keras extension
        model.save(save_path)
        logger.log("✅ Model saved successfully!")
    except Exception as e:
        logger.log(f"❌ Save failed: {e}", "ERROR")
        raise

def load_model_tf(save_dir, logger):
    try:
        model_path = os.path.join(save_dir, "model.keras")
        if not os.path.exists(model_path):
            logger.log("❌ No saved model found.", "ERROR")
            return None
        logger.log(f"ℹ️ Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path)
        logger.log("✅ Model loaded successfully!")
        return model
    except Exception as e:
        logger.log(f"❌ Load failed: {e}", "ERROR")
        return None

# ---------------- TRAINING THREAD ----------------
def train_models_thread(symbol):
    TRAINING_FLAGS[symbol] = True
    logger = Logger(symbol)
    TRAIN_LOGS.setdefault(symbol, [])
    try:
        logger.log(f"🚀 Starting training for {symbol}")
        df = download_stock_data(symbol, period='1y', logger=logger)
        df = add_technical_indicators(df, logger=logger)
        X, y, scaler, n_features = prepare_sequences(df, SEQ_LEN, logger=logger)
        if len(X) < 100:
            raise ValueError(f"Insufficient data: {len(X)} sequences (need 100+ for reliable training)")
        split_idx = int(len(X) * 0.8)
        X_train, X_test = X[:split_idx], X[split_idx:]
        y_train, y_test = y[:split_idx], y[split_idx:]
        logger.log(f"Training: {len(X_train)} samples, Testing: {len(X_test)} samples")
        # Train LSTM
        logger.log("🏗️ Building LSTM model...")
        lstm_model = build_lstm_model(SEQ_LEN, n_features)
        logger.log(f"🎓 Training LSTM ({EPOCHS} epochs)...")
        history_lstm = lstm_model.fit(
            X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=EPOCHS,
            batch_size=BATCH_SIZE,
            verbose=0,
            callbacks=[callbacks.EarlyStopping(patience=2, restore_best_weights=True)]
        )
        final_loss = history_lstm.history.get('val_loss', [None])[-1]
        logger.log(f"LSTM trained! Val Loss: {final_loss:.6f}", "SUCCESS")
        lstm_dir = os.path.join(MODELS_DIR, f"{symbol}_lstm_saved_model")
        save_model_tf(lstm_model, lstm_dir, logger)
        # Train Bidirectional LSTM
        logger.log("🏗️ Building Bidirectional LSTM model...")
        bilstm_model = build_bidirectional_lstm_model(SEQ_LEN, n_features)
        logger.log(f"🎓 Training Bidirectional LSTM ({EPOCHS} epochs)...")
        history_bilstm = bilstm_model.fit(
            X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=EPOCHS,
            batch_size=BATCH_SIZE,
            verbose=0,
            callbacks=[callbacks.EarlyStopping(patience=2, restore_best_weights=True)]
        )
        final_loss2 = history_bilstm.history.get('val_loss', [None])[-1]
        logger.log(f"Bidirectional LSTM trained! Val Loss: {final_loss2:.6f}", "SUCCESS")
        bilstm_dir = os.path.join(MODELS_DIR, f"{symbol}_bilstm_saved_model")
        save_model_tf(bilstm_model, bilstm_dir, logger)
        # Save scaler
        scaler_path = os.path.join(MODELS_DIR, f"{symbol}_scaler.joblib")
        joblib.dump(scaler, scaler_path)
        logger.log(f"Scaler saved to {scaler_path}", "SUCCESS")
        # Zip saved models for download
        for d in (lstm_dir, bilstm_dir):
            base = d
            zip_path = f"{base}.zip"
            if os.path.exists(zip_path):
                os.remove(zip_path)
            shutil.make_archive(base, 'zip', base)
            logger.log(f"Created archive {zip_path}", "INFO")
        logger.log("🎉 Training completed successfully!", "SUCCESS")
    except Exception as e:
        logger.log(f"Training failed: {str(e)}", "ERROR")
        tb = traceback.format_exc()
        for line in tb.splitlines():
            logger.log(line, "ERROR")
    finally:
        TRAINING_FLAGS[symbol] = False

# ---------------- HELPERS ----------------
def is_trained(symbol):
    lstm_dir = os.path.join(MODELS_DIR, f"{symbol}_lstm_saved_model")
    bilstm_dir = os.path.join(MODELS_DIR, f"{symbol}_bilstm_saved_model")
    scaler_path = os.path.join(MODELS_DIR, f"{symbol}_scaler.joblib")
    return os.path.exists(lstm_dir) and os.path.exists(bilstm_dir) and os.path.exists(scaler_path)

def load_models_and_scaler(symbol):
    logger = Logger(symbol)
    lstm_dir = os.path.join(MODELS_DIR, f"{symbol}_lstm_saved_model")
    bilstm_dir = os.path.join(MODELS_DIR, f"{symbol}_bilstm_saved_model")
    scaler_path = os.path.join(MODELS_DIR, f"{symbol}_scaler.joblib")
    lstm_model = load_model_tf(lstm_dir, logger)
    bilstm_model = load_model_tf(bilstm_dir, logger)
    if lstm_model is None or bilstm_model is None:
        raise RuntimeError("Failed to load models")
    if not os.path.exists(scaler_path):
        raise FileNotFoundError("Scaler file not found")
    scaler = joblib.load(scaler_path)
    return lstm_model, bilstm_model, scaler

def ensemble_predict(lstm_model, bilstm_model, X):
    p1 = lstm_model.predict(X, verbose=0).flatten()
    p2 = bilstm_model.predict(X, verbose=0).flatten()
    return (p1 + p2) / 2.0

# ---------------- UI FUNCTIONS ----------------
def start_training(symbol):
    if TRAINING_FLAGS.get(symbol, False):
        return f"⚠️ Training already running for {symbol}"
    TRAIN_LOGS[symbol] = []
    thread = threading.Thread(target=train_models_thread, args=(symbol,), daemon=True)
    thread.start()
    return f"🚀 Started training for {symbol}. Click 'Refresh Logs' to view progress."

def get_training_logs(symbol):
    return logs_to_text(symbol)

def make_prediction(symbol):
    try:
        if not is_trained(symbol):
            return f"❌ Models not trained for {symbol}. Train first!"
        lstm_model, bilstm_model, scaler = load_models_and_scaler(symbol)
        df = download_stock_data(symbol, period='3mo')
        df = add_technical_indicators(df)
        X, y, _, _ = prepare_sequences(df, SEQ_LEN)
        if len(X) == 0:
            return "❌ Insufficient data for prediction"
        last_seq = X[-1:]
        pred_scaled = ensemble_predict(lstm_model, bilstm_model, last_seq)[0]
        n_features = scaler.scale_.shape[0]
        dummy = np.zeros((1, n_features), dtype=np.float32)
        dummy[0, 0] = pred_scaled
        pred_price = scaler.inverse_transform(dummy)[0, 0]
        current_price = df['Close'].values[-1]
        change_pct = ((pred_price - current_price) / current_price) * 100
        result = (
            f"📊 Prediction for {symbol}\n\n"
            f"Current Price: ${current_price:.2f}\n"
            f"Predicted Next: ${pred_price:.2f}\n"
            f"Expected Change: {change_pct:+.2f}%\n\n"
            f"🤖 Ensemble: LSTM + Bidirectional LSTM"
        )
        return result
    except Exception as e:
        return f"❌ Error during prediction: {str(e)}"

def get_validation_plot(symbol):
    try:
        plt.close('all')  # clear previous figures
        if not is_trained(symbol):
            fig, ax = plt.subplots(figsize=(10, 4))
            ax.text(0.5, 0.5, f"Models not trained for {symbol}\nClick 'Train Models' first", ha='center', va='center', fontsize=14)
            ax.axis('off')
            return fig
        lstm_model, bilstm_model, scaler = load_models_and_scaler(symbol)
        df = download_stock_data(symbol, period='6mo')
        df = add_technical_indicators(df)
        X, y, _, _ = prepare_sequences(df, SEQ_LEN)
        if len(X) < 2:
            fig, ax = plt.subplots(figsize=(10, 4))
            ax.text(0.5, 0.5, "Insufficient data for validation plot", ha='center', va='center', fontsize=12)
            ax.axis('off')
            return fig
        split_idx = int(len(X) * 0.8)
        X_test = X[split_idx:]
        y_test = y[split_idx:]
        pred_scaled = ensemble_predict(lstm_model, bilstm_model, X_test)
        n_features = scaler.scale_.shape[0]
        dummy_pred = np.zeros((len(pred_scaled), n_features), dtype=np.float32)
        dummy_pred[:, 0] = pred_scaled
        pred_prices = scaler.inverse_transform(dummy_pred)[:, 0]
        dummy_actual = np.zeros((len(y_test), n_features), dtype=np.float32)
        dummy_actual[:, 0] = y_test
        actual_prices = scaler.inverse_transform(dummy_actual)[:, 0]
        fig, ax = plt.subplots(figsize=(12, 5))
        ax.plot(actual_prices, label='Actual', linewidth=2)
        ax.plot(pred_prices, label='Predicted', linewidth=2, alpha=0.7)
        ax.set_title(f'{symbol} - Validation: Actual vs Predicted', fontsize=14, fontweight='bold')
        ax.set_xlabel('Time Steps')
        ax.set_ylabel('Price ($)')
        ax.legend()
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        return fig
    except Exception as e:
        fig, ax = plt.subplots(figsize=(10, 4))
        ax.text(0.5, 0.5, f"Error generating plot:\n{str(e)}", ha='center', va='center', fontsize=12)
        ax.axis('off')
        return fig

def get_head_of_data(symbol):
    """Return df.head(10) after indicators (3 months of data) as a pandas DataFrame"""
    try:
        df = download_stock_data(symbol, period='3mo')
        df = add_technical_indicators(df)
        # ensure we return a DataFrame
        return df.head(10)
    except Exception:
        # return empty DataFrame on failure
        return pd.DataFrame()

def download_model_files(symbol):
    if not is_trained(symbol):
        return None, None, None
    lstm_dir = os.path.join(MODELS_DIR, f"{symbol}_lstm_saved_model")
    bilstm_dir = os.path.join(MODELS_DIR, f"{symbol}_bilstm_saved_model")
    scaler_path = os.path.join(MODELS_DIR, f"{symbol}_scaler.joblib")
    lstm_zip = f"{lstm_dir}.zip"
    bilstm_zip = f"{bilstm_dir}.zip"
    if not os.path.exists(lstm_zip):
        shutil.make_archive(lstm_dir, 'zip', lstm_dir)
    if not os.path.exists(bilstm_zip):
        shutil.make_archive(bilstm_dir, 'zip', bilstm_dir)
    return lstm_zip, bilstm_zip, scaler_path

# ---------------- GRADIO UI ----------------
with gr.Blocks(theme=gr.themes.Soft(), title="Stock Predictor") as app:
    def get_dataset_preview(symbol):
        try:
            df = download_stock_data(symbol, period='6mo')
            df = add_technical_indicators(df)
            return df.head(10)
        except Exception as e:
            return pd.DataFrame({"Error": [str(e)]})

    with gr.Row():
        stock_dropdown = gr.Dropdown(choices=TOP_10_STOCKS, value="AAPL", label="📊 Select Stock", interactive=True)
        train_button = gr.Button("🚀 Train Models", variant="primary", size="lg")
    with gr.Row():
        logs_display = gr.Textbox(label="📋 Training Logs", lines=12, max_lines=20, interactive=False)
        refresh_logs_btn = gr.Button("🔄 Refresh Logs")
    gr.Markdown("---")
    gr.Markdown("## 🎯 Make Predictions")
    with gr.Row():
        predict_button = gr.Button("🔮 Predict Next Price", variant="secondary", size="lg")
        prediction_output = gr.Textbox(label="Prediction Result", lines=7, interactive=False)
    with gr.Row():
        validation_plot = gr.Plot(label="📊 Validation Plot (Actual vs Predicted)")
    gr.Markdown("---")
    gr.Markdown("## 🗒️ Dataset Preview (head(10))")
    with gr.Row():
      df_preview = gr.Dataframe(label="Dataset Preview (first 10 rows)", interactive=False)

    gr.Markdown("---")
    gr.Markdown("## 💾 Download Trained Models")
    with gr.Row():
        lstm_download = gr.File(label="LSTM Model (ZIP)")
        bilstm_download = gr.File(label="Bidirectional LSTM Model (ZIP)")
        scaler_download = gr.File(label="Scaler Data")
    # Events
    train_button.click(fn=start_training, inputs=[stock_dropdown], outputs=[logs_display])
    refresh_logs_btn.click(fn=get_training_logs, inputs=[stock_dropdown], outputs=[logs_display])
    predict_button.click(fn=make_prediction, inputs=[stock_dropdown], outputs=[prediction_output])
    # ensure plot updates when clicking Predict
    predict_button.click(fn=get_validation_plot, inputs=[stock_dropdown], outputs=[validation_plot])
    # dataset head updates on Predict and when changing dropdown
    predict_button.click(fn=get_head_of_data, inputs=[stock_dropdown], outputs=[df_preview])
    stock_dropdown.change(fn=get_head_of_data, inputs=[stock_dropdown], outputs=[df_preview])
    # update downloads when stock selection changes
    stock_dropdown.change(fn=download_model_files, inputs=[stock_dropdown], outputs=[lstm_download, bilstm_download, scaler_download])

if __name__ == "__main__":
    print("=" * 50)
    print("🚀 Stock Predictor App Starting...")
    print("=" * 50)
    app.launch(share=True, debug=False)

🚀 Stock Predictor App Starting...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9fee59a26c8cac8792.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
